In [6]:
# relevant imports 
import pandas as pd
import random
import uuid


In [7]:
# Potential Categories of tests that ZAP might perform / Team 2's choice 
ZAP_TEST_CATEGORIES = [
    "Spider Scan", "Passive Scan", "Active Scan",
    "SQL Injection", "XSS", "Security Headers",
    "Broken Auth", "Directory Traversal"
]


In [8]:
#Simulates vulnerability types that OWASP ZAP might detect, each paired with a risk level.
ZAP_ALERT_POOL = [
    {
        "name": "SQL Injection",
        "risk": "High",
        "description": "The application appears to be vulnerable to SQL injection attacks, which allow attackers to manipulate backend queries.",
        "solution": "Use parameterized queries or prepared statements to handle user input safely."
    },
    {
        "name": "Cross-Site Scripting (XSS)",
        "risk": "High",
        "description": "The application reflects unsanitized user input back to the browser, allowing JavaScript injection.",
        "solution": "Sanitize and encode all user-generated content before rendering it on the page."
    },
    {
        "name": "Remote Code Execution (RCE)",
        "risk": "High",
        "description": "An attacker may be able to execute arbitrary commands on the server due to insecure input handling.",
        "solution": "Validate and sanitize inputs. Use secure frameworks that don't pass user input to system commands."
    },
    {
        "name": "Content Security Policy Not Set",
        "risk": "Medium",
        "description": "The application is missing a Content-Security-Policy header which can mitigate XSS and other attacks.",
        "solution": "Implement a strong Content-Security-Policy header to restrict resource loading."
    },
    {
        "name": "Missing X-Frame-Options Header",
        "risk": "Medium",
        "description": "The application is missing the X-Frame-Options header, making it potentially vulnerable to clickjacking attacks.",
        "solution": "Add the X-Frame-Options: SAMEORIGIN header to all responses."
    },
    {
        "name": "Insecure Cookie - No Secure Flag",
        "risk": "Low",
        "description": "A cookie is being set without the Secure flag, meaning it could be transmitted over unencrypted connections.",
        "solution": "Ensure all sensitive cookies are marked as Secure to enforce HTTPS transmission."
    },
    {
        "name": "Insecure Cookie - No HttpOnly Flag",
        "risk": "Low",
        "description": "A cookie is accessible via JavaScript, increasing the risk of theft via XSS.",
        "solution": "Mark cookies with the HttpOnly flag to prevent client-side access."
    },
    {
        "name": "Information Disclosure - Debug Error Messages",
        "risk": "Informational",
        "description": "Error messages revealing stack traces or system paths are being shown to users.",
        "solution": "Disable verbose error output in production. Use custom error pages instead."
    },
    {
        "name": "Directory Browsing Enabled",
        "risk": "Informational",
        "description": "The web server allows directory listing, which may expose sensitive files.",
        "solution": "Disable directory browsing on the server configuration."
    },
    {
        "name": "Server Leaks Version Information via HTTP Headers",
        "risk": "Informational",
        "description": "The server reveals its version in headers like Server or X-Powered-By.",
        "solution": "Configure the server to hide or remove version headers."
    }
]


In [9]:
# Severity buckets to categorize and count how many alerts fall into each severity level
SEVERITY_RATINGS = ["High", "Medium", "Low", "Informational"]

In [ ]:
# Main function to generate synthetic test data
def generate_synthetic_zap_test_cases(num_cases=200, seed=123):
    if seed is not None:
        random.seed(seed)  # For reproducibility

    zap_cases = []

    for i in range(1, num_cases + 1):
        # Simulate identifiers
        test_case_id = f"ZAP-TC-{i:04}" # e.g., ZAP-TC-0001
        project_id = f"PRJ-{random.randint(1, 50):03}" # Random project ID like PRJ-007
        user_id = f"USER-{random.randint(1, 50):03}" # Random user ID
        test_category = random.choice(ZAP_TEST_CATEGORIES) # Pick a random category from the test categories 
        
        # Generate a fake URL using UUID for uniqueness
        target_url = f"https://example.com/{uuid.uuid4().hex[:6]}" # Fake unique URL
        
        # Pick a random number of alerts from the alert pool
        num_alerts = random.randint(2, 6)
        alerts = random.choices(ZAP_ALERT_POOL, k=num_alerts)

        # Count the number of alerts by severity
        severity_counts = {severity: 0 for severity in SEVERITY_RATINGS} # Initialize severity counts
        for alert in alerts:
            severity_counts[alert["risk"]] += 1

        # Determine test result: if any High severity alert → Fail, else Pass
        result_status = "Passed" if severity_counts["High"] == 0 else "Failed"

        # Build one synthetic test case row in a dictionary 
        zap_case = {
            "Project ID": project_id,
            "User ID": user_id,
            "Test Case ID": test_case_id,
            "Test Category": test_category,
            "Target URL": target_url,
            "Alerts Detected": num_alerts,
            "High": severity_counts["High"],
            "Medium": severity_counts["Medium"],
            "Low": severity_counts["Low"],
            "Informational": severity_counts["Informational"],
            "Result": result_status
        }

        zap_cases.append(zap_case)
# Convert list of dictionaries into a DataFrame for easy analysis/export
    return pd.DataFrame(zap_cases)

In [ ]:
# Run and display results
df_zap_synthetic = generate_synthetic_zap_test_cases()
df_zap_synthetic.head() # Print first 5 rows to inspect

#Saving as CSV 
df_zap_synthetic.to_csv("synthetic_zap_test_cases.csv", index=False)
print("\nSaved to 'synthetic_zap_test_cases.csv'")



,Project ID,User ID,Test Case ID,Test Category,Target URL,Alerts Detected,High,Medium,Low,Informational,Result
0,PRJ-004,USER-018,ZAP-TC-0001,Passive Scan,https://example.com/756c20,5,1,1,1,2,Failed
1,PRJ-022,USER-004,ZAP-TC-0002,Active Scan,https://example.com/5bfd25,3,1,2,0,0,Failed
2,PRJ-001,USER-028,ZAP-TC-0003,Passive Scan,https://example.com/fc942d,6,3,1,1,1,Failed
3,PRJ-009,USER-002,ZAP-TC-0004,XSS,https://example.com/12b143,5,1,1,1,2,Failed
4,PRJ-031,USER-014,ZAP-TC-0005,Security Headers,https://example.com/cfb213,2,0,1,0,1,Passed
